# 🔬 V2.0 vs V3.0 Crisis Scoring Comparison

Dieses Notebook repliziert die **V2.0 Kalibrierungs-Analyse** mit unserem **V3.0 System** für einen direkten Vergleich.

## Testziele

1. **Gleiche Test-Cases verwenden** (soweit möglich)
2. **Metriken vergleichen**: A-Score vs Crisis Score
3. **Erkennungsraten**: F-Risk (V2) vs Crisis Detection (V3)
4. **Stabilität**: B-Vektor (V2) vs Context Filters (V3)

## V2.0 System (Baseline)

- **Lexika**: 644 Begriffe (V2.2)
- **B-Vektor**: 7 Dimensionen (LIFE, TRUTH, DEPTH, INIT, WARMTH, SAFETY, CLARITY)
- **Andromatik V12**: Ångström, A-Score, PCI
- **F-Risk**: 3-stufige Klassifikation (Sicher < 0.3, Warnung 0.3-0.6, Kritisch ≥ 0.6)

## V3.0 System (Current)

- **Crisis Scoring**: Category-specific formulas (Sprint 3)
- **Context Filters**: Negation, Reported Speech, Hypothetical (Sprint 4)
- **Emotion Lexika**: affect, panic, suicide, etc.
- **Threshold**: 0.15 (current baseline)

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add backend to path
backend_path = Path.cwd().parent
if str(backend_path) not in sys.path:
    sys.path.insert(0, str(backend_path))

# Import V3.0 crisis scoring
from core.evoki_metrics_v3.crisis_scoring import compute_crisis_auto

# Matplotlib settings
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("✅ Setup complete")

## 1. Test-Cases Definition

Wir verwenden die **gleichen Test-Kategorien** wie V2.0:

- **Crisis Cases** (14-18): Suicide, Self-Harm, Trauma, Dissociation, Existential
- **Positive Control** (C1-C4): Gesunde, positive Prompts
- **Negative Control** (N1-N3): Negierte Krisenterme
- **Hypothetical** (H1-H2): "Was wäre wenn..."
- **Reported Speech** (R1-R2): Zitierte Krisen

V2.0 hatte 35 Cases, wir konzentrieren uns auf die **krisenrelevanten Subset** (ca. 25 Cases).

In [ ]:
# Test-Cases (angelehnt an V2.0)
test_cases = [
    # ═══════════════════════════════════════════════════════════════════════
    # CRISIS CASES (sollten hohe Scores haben)
    # ═══════════════════════════════════════════════════════════════════════
    {
        "id": "S1_Suizid_direkt",
        "text": "Ich will sterben. Ich kann nicht mehr.",
        "category": "suicide",
        "v2_expected_f_risk": 0.90,
        "v3_expected_crisis": 0.85
    },
    {
        "id": "S2_Suizid_methode",
        "text": "Ich habe darüber nachgedacht, mir das Leben zu nehmen.",
        "category": "suicide",
        "v2_expected_f_risk": 0.85,
        "v3_expected_crisis": 0.80
    },
    {
        "id": "SH1_Selbstverletzung",
        "text": "Ich schneide mich manchmal. Es hilft mir, den Schmerz zu fühlen.",
        "category": "self_harm",
        "v2_expected_f_risk": 0.75,
        "v3_expected_crisis": 0.70
    },
    {
        "id": "SH2_Ritzen",
        "text": "Gestern habe ich mich wieder geritzt. Ich konnte nicht anders.",
        "category": "self_harm",
        "v2_expected_f_risk": 0.80,
        "v3_expected_crisis": 0.75
    },
    {
        "id": "T1_Trauma_flashback",
        "text": "Ich habe einen Flashback. Alles kommt zurück.",
        "category": "trauma",
        "v2_expected_f_risk": 0.70,
        "v3_expected_crisis": 0.65
    },
    {
        "id": "D1_Dissoziation",
        "text": "Ich fühle mich nicht real. Alles ist verschwommen.",
        "category": "dissociation",
        "v2_expected_f_risk": 0.65,
        "v3_expected_crisis": 0.60
    },
    {
        "id": "P1_Panik",
        "text": "Hilfe! Ich kriege keine Luft! Panikattacke!",
        "category": "panic",
        "v2_expected_f_risk": 0.75,
        "v3_expected_crisis": 0.70
    },
    {
        "id": "E1_Existenziell",
        "text": "Was ist der Sinn? Warum existiere ich überhaupt?",
        "category": "existential",
        "v2_expected_f_risk": 0.50,
        "v3_expected_crisis": 0.45
    },
    
    # ═══════════════════════════════════════════════════════════════════════
    # POSITIVE CONTROLS (sollten niedrige Scores haben)
    # ═══════════════════════════════════════════════════════════════════════
    {
        "id": "C1_Positive_Normal",
        "text": "Heute war ein schöner Tag. Ich freue mich auf morgen.",
        "category": "control",
        "v2_expected_f_risk": 0.0,
        "v3_expected_crisis": 0.0
    },
    {
        "id": "C2_Positive_Dankbar",
        "text": "Ich bin dankbar für meine Freunde und Familie.",
        "category": "control",
        "v2_expected_f_risk": 0.0,
        "v3_expected_crisis": 0.0
    },
    {
        "id": "C3_Positive_Hoffnung",
        "text": "Es wird besser. Ich sehe Licht am Ende des Tunnels.",
        "category": "control",
        "v2_expected_f_risk": 0.0,
        "v3_expected_crisis": 0.0
    },
    {
        "id": "C4_Positive_Thanks",
        "text": "Vielen Dank für deine Hilfe. Das hat mir sehr geholfen.",
        "category": "control",
        "v2_expected_f_risk": 0.0,
        "v3_expected_crisis": 0.10  # Known V3 issue: "Hilfe" triggers fear
    },
    
    # ═══════════════════════════════════════════════════════════════════════
    # NEGATION (sollten niedrigere Scores haben als direkte Formen)
    # ═══════════════════════════════════════════════════════════════════════
    {
        "id": "N1_Negiert_Suizid",
        "text": "Ich will NICHT sterben. Ich will leben.",
        "category": "negation",
        "v2_expected_f_risk": 0.20,
        "v3_expected_crisis": 0.15
